<a href="https://colab.research.google.com/github/angiew-02/AHI-Capstone-Project/blob/main/AHI_Capstone_25'_Tableau_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing libraries
!pip install seaborn
!pip install pandas
!pip install numpy
!pip3 install pandas
!pip install matplotlib
!pip install xgboost
!pip install catboost
!pip install regex
!pip install scikit-learn
!pip install lightgbm
!pip install missingno
!pip install plotly.express

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
#import statsmodels.api as sm
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split  # For splitting the dataset into training and testing sets
from sklearn.linear_model import LinearRegression  # Linear regression model
from sklearn.metrics import mean_squared_error, r2_score  # Evaluation metrics
import matplotlib.pyplot as plt  # For data visualization

In [6]:
heartdisease = pd.read_csv('/content/Rates_and_Trends_in_Heart_Disease_and_Stroke_Mortality_Among_US_Adults__35___by_County__Age_Group__Race_Ethnicity__and_Sex___2000-2019.csv')

/tmp/ipython-input-6-2955419739.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  heartdisease = pd.read_csv('/content/Rates_and_Trends_in_Heart_Disease_and_Stroke_Mortality_Among_US_Adults__35___by_County__Age_Group__Race_Ethnicity__and_Sex___2000-2019.csv')


In [7]:
print(heartdisease.columns)

Index(['Year', 'LocationAbbr', 'LocationDesc', 'GeographicLevel', 'DataSource',
       'Class', 'Topic', 'Data_Value', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Confidence_limit_Low', 'Confidence_limit_High',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'StratificationCategory3', 'Stratification3',
       'LocationID'],
      dtype='object')


In [8]:
print(heartdisease.dtypes)

Year                           object
LocationAbbr                   object
LocationDesc                   object
GeographicLevel                object
DataSource                     object
Class                          object
Topic                          object
Data_Value                    float64
Data_Value_Unit                object
Data_Value_Type                object
Data_Value_Footnote_Symbol     object
Data_Value_Footnote            object
Confidence_limit_Low          float64
Confidence_limit_High         float64
StratificationCategory1        object
Stratification1                object
StratificationCategory2        object
Stratification2                object
StratificationCategory3        object
Stratification3                object
LocationID                      int64
dtype: object


In [9]:
heartdisease.replace('?', np.nan, inplace=True)

In [10]:
heartdisease.drop_duplicates(inplace=True)

In [11]:
# Replace empty strings or strings with only whitespace with np.nan
heartdisease = heartdisease.replace(r'^\s*$', np.nan, regex=True)

In [12]:
columns_to_drop = [
    'StratificationCategory1', 'StratificationCategory2', 'StratificationCategory3',
    'DataSource','GeographicLevel','Class','Data_Value_Unit','Data_value_Type',
    'Data_Value_Footnote_Symbol','Data_Value_Footnote'
]
# Drop only existing columns
existing_cols = [col for col in columns_to_drop if col in heartdisease.columns]
heartdisease_cleaned = heartdisease.drop(columns=existing_cols)
print(heartdisease_cleaned.head(10))

   Year LocationAbbr LocationDesc              Topic  Data_Value  \
0  1999           AL      Autauga  All heart disease         NaN   
1  2013           AL      Autauga  All heart disease         NaN   
2  2014           AL      Autauga  All heart disease         NaN   
3  2005           AL      Autauga  All heart disease         NaN   
4  2012           AL      Autauga  All heart disease         NaN   
5  2010           AL      Autauga  All heart disease         NaN   
6  2009           AL      Autauga  All heart disease         NaN   
7  2011           AL      Autauga  All heart disease         NaN   
8  2007           AL      Autauga  All heart disease         NaN   
9  2019           AL      Autauga  All heart disease         NaN   

                                    Data_Value_Type  Confidence_limit_Low  \
0  Age-Standardized, Spatiotemporally Smoothed Rate                   NaN   
1  Age-Standardized, Spatiotemporally Smoothed Rate                   NaN   
2  Age-Standardized,

In [13]:
# Define the mapping gender to numeric midpoints
# Step 3: Convert gender to numerical midpoints
Stratification3_mapping = {
    'Overall': 5,
    'Male': 15,
    'Female': 25,
}
heartdisease_cleaned['Stratification3'] = heartdisease_cleaned['Stratification3'].map(Stratification3_mapping)
# Preview to confirm
heartdisease_cleaned[['Stratification3']].head(20)

,Stratification3
0,5.0
1,5.0
2,5.0
3,5.0
4,5.0
5,5.0
6,5.0
7,5.0
8,5.0
9,5.0


In [14]:
# Data preview
heartdisease_cleaned.head(20)

,Year,LocationAbbr,LocationDesc,Topic,Data_Value,Data_Value_Type,Confidence_limit_Low,Confidence_limit_High,Stratification1,Stratification2,Stratification3,LocationID
0,1999,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
1,2013,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
2,2014,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
3,2005,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
4,2012,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
5,2010,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
6,2009,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
7,2011,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
8,2007,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
9,2019,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001


In [16]:
print(heartdisease_cleaned.columns)

Index(['Year', 'LocationAbbr', 'LocationDesc', 'Topic', 'Data_Value',
       'Data_Value_Type', 'Confidence_limit_Low', 'Confidence_limit_High',
       'Stratification1', 'Stratification2', 'Stratification3', 'LocationID'],
      dtype='object')


In [17]:
# Preview
heartdisease_cleaned.head(10)

,Year,LocationAbbr,LocationDesc,Topic,Data_Value,Data_Value_Type,Confidence_limit_Low,Confidence_limit_High,Stratification1,Stratification2,Stratification3,LocationID
0,1999,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
1,2013,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
2,2014,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
3,2005,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
4,2012,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
5,2010,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
6,2009,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
7,2011,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
8,2007,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001
9,2019,AL,Autauga,All heart disease,NaN,"Age-Standardized, Spatiotemporally Smoothed Rate",NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,5.0,1001


In [19]:
# Check for missing values in the specified columns
columns_to_check = ['Year','LocationAbbr','LocationDesc','Topic','Data_Value','Data_Value_Type','Confidence_limit_Low','Confidence_limit_High','Stratification1','Stratification2','Stratification3','LocationID']

# Display missing values count for each column
missing_values = heartdisease_cleaned[columns_to_check].isnull().sum()
print("Missing values in selected columns:")
print(missing_values)

Missing values in selected columns:
Year                           0
LocationAbbr                   0
LocationDesc                   0
Topic                          0
Data_Value               2365475
Data_Value_Type                0
Confidence_limit_Low     2365475
Confidence_limit_High    2365475
Stratification1                0
Stratification2                0
Stratification3          1442560
LocationID                     0
dtype: int64
